In [89]:
from IPython.core.debugger import Tracer
from scipy.io import wavfile
set_trace = Tracer()

c:\users\jonny.wong\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel\__main__.py:3: DeprecationWarning: `Tracer` is deprecated since version 5.1, directly use `IPython.core.debugger.Pdb.set_trace()`
  app.launch_new_instance()


In [3]:
import numpy as np
%matplotlib
from pylab import plot
pi = np.pi
e = np.e
fft = np.fft.fft
sin = np.sin

Using matplotlib backend: TkAgg


In [2]:
def get5(a):
    return a[110:115]

In [4]:
a = np.arange(0,1,1/44100)
fs = 44.1*1000
framesize = 2048
fb = fs/framesize

In [5]:
fres1 = fb*112
fres2 = fb*112.2
fres3 = fb*112.49
fres4 = fb*112.50

In [6]:
class Ps:
    def __init__(self, freq):
        for i in range(5):
            key = 'c'+str(i)
            data = sin(2*pi*freq*a[2048*i:2048*(i+1)])
            self.__dict__.update({key:data})
            
            key = 'fft'+str(i)
            fft_data = fft(data)
            self.__dict__.update({key: fft_data})  

In [7]:
def get5ang(d):
    ang = []
    for i in range(5):
        key = 'fft'+str(i)
        data = d.__dict__[key][110:115]
        ang.append(np.angle(data))
    return ang

def diff(ang):
    return [np.round(ang[i+1]-ang[i],6) for i in range(len(ang)-1)]

In [117]:
gtmp = []

def norm2pi(phase):
    # about qpd is:
    # -7 to -8, 7 to 8;
    # -4 to -4, 4 to 4;
    qpd = (phase/pi).astype(int)
    qpd += np.sign(qpd)*(qpd&1)
    return phase - pi*qpd # here phase is in -pi~pi

def ana(audioData, spRate=44100, frameSize=4096, ovrSmp=4, pitchShift=1):   
    stepSize = frameSize//ovrSmp
    freqPerBin = spRate/frameSize
    expct = 2.0*pi*(stepSize/frameSize)
    
    # split the data
    blocks = len(audioData)//stepSize
    dataFrames = [audioData[stepSize*n:(stepSize*n+frameSize)] for n in range(blocks)]
    rsltFrames = [0.]*blocks
    gSumPhase = np.array([0.]*frameSize)
    gOutputAccum = np.array([0.]*frameSize)
    
    # 窗函数
    k_range = np.arange(0,frameSize,1)
    # window = .5*(1-np.cos(2.*pi*k_range/(frameSize-1)))
    window = 0.53836 - 0.46164*np.cos(2.*pi*k_range/(frameSize-1))
    # window = e**(-0.5*((k_range-(frameSize-1)/2)/(0.5*(frameSize-1)/2))**2)
    
    # analysis
    lastPhase = np.array([0.]*frameSize)
    for idx, frame in enumerate(dataFrames):
        # 加窗
        fftData = fft(frame*window)
        
        #########
        #break
        global gtmp
        #gtmp = fftData
        #plot(frame*window)
        #return
        #########       
        
        # 幅值之所以乘以2，是因为将频谱后半部分归零，ifft之后幅值降为一半
        # 要不然的话，就得把频谱前半部分镜像对称到后半部分，不如归零再乘2
        # 来的方便
        magn = 2*abs(fftData)
        phase = np.angle(fftData)
        
        tmp = phase - lastPhase
        lastPhase = phase.copy()
        
        tmp -= k_range*expct

        thePhase = tmp.copy()
        theMagn = magn.copy()

        # about qpd is:
        # -7 to -8, 7 to 8;
        # -4 to -4, 4 to 4;
        #qpd = (tmp/pi).astype(int)
        #qpd += np.sign(qpd)*(qpd&1)
        #tmp -= pi*qpd # here tmp is in -pi~pi
        tmp = norm2pi(tmp)
        
        tmp = ovrSmp*tmp/(2*pi)

        theFreq = (k_range + tmp)*freqPerBin

        #===============================
        # We got theFreq theMagn and thePhase here.
        # Do something with theFreq, and then synthesis
        # new complex array (with Euler's formula：
        # magn * e ** (angle * pi))

        lambda x:x(theFreq)

        #set_trace()
        # Calc new phase
        tmp = theFreq.copy()
        tmp = tmp/freqPerBin - k_range
        tmp = 2*pi*ovrSmp + k_range*expct
       
        gSumPhase += tmp
        thePhase = gSumPhase.copy()
        
        k = np.append(np.ones(frameSize//2), np.zeros(frameSize//2))
        theFFT = theMagn*e**(thePhase*1j) * k
        rawFrame = np.real( np.fft.ifft(theFFT) )
        
        #plot(rawFrame)
        x = rawFrame[2048:]
        y = rawFrame[:2048]
        z = np.append(x,y)
        plot(z)
        plot(frame*window)
        
        gOutputAccum += 2*window*rawFrame
        stepFrame = gOutputAccum[:stepSize]
        
        
        return theFreq
    
    return np.concatenate(rsltFrames)

In [118]:
fq = 123
x = np.arange(0,1,1/44100)
y = np.sin(2*pi*fq*x)+0.8*np.sin(2*pi*fq*2*x)+0.64*np.sin(2*pi*fq*4*x)
#plot(x,y)

In [119]:
ay = ana(y, ovrSmp=1, pitchShift = 1.25)

In [122]:
e**(1j)

(0.5403023058681398+0.8414709848078965j)

In [121]:
np.cos(1)+1j*sin(1)

(0.54030230586813977+0.8414709848078965j)